Last updated: Jan 5th 2016

# Climate data exploration: a journey through Pandas: part 2

In the part 1, we (down)loaded datasets for temperature, and sea level measurements, cleaned them up and visualized them. In this second part, we will analyze these datasets and get to answer the questions we started with:
  - How correlated are the global temperature, sea levels and greenhouse gaz emission (of course, correlation doesn't prove causation)?
  - What temperature and sea level rise should we expect for the end of the century in New York (or any other coastal city in the world you are interested in)?

In the process, we will learn about:

Part 1:

    1. Loading data
    2. Pandas datastructures
    3. Cleaning and formatting data
    4. Statistical description of data 
    5. Basic visualization
    6. Exporting data to files
    
Part 2:
    
    7. Accessing data
    8. Working with dates and times
    9. Transforming datasets
    10. Data agregation and summarization
    11. Correlations and regressions
    12. Predictions from auto regression models

## Initial setup (again)

In [ ]:
%matplotlib inline
import pandas as pd
print "Pandas version:", pd.__version__

import numpy as np
import matplotlib.pyplot as plt

# This allows to control a lot of styling options, which might be
# quite critical when working with data:
pd.set_option("display.max_rows", 16)

# This will be useful to blow matplotlib figures larger:
LARGE_FIGSIZE = (12, 8)

In [ ]:
%ls

Let's reload the clean data that we stored in part 1:

In [ ]:
# If part1 failed to generate a clean dataset, load the backup file:
# store = pd.HDFStore("data/cleaned_climate_data_BACKUP.h5")
store = pd.HDFStore("cleaned_climate_data.h5", "r")
mean_sea_level = pd.read_hdf(store, "/sea_level/timeseries")
local_sea_level_stations = pd.read_hdf(store, "/sea_level/station_data")
giss_temp = pd.read_hdf(store, "/temperature/monthly")
full_globe_temp = pd.read_hdf(store, "/temperature/yearly")
store.close()

In [ ]:
## If there are problems with Pandas and the PyTables package, load
## results from backup CSV files instead.
# mean_sea_level = pd.read_csv("data/clean_data/sea_level.csv", index_col=0)
# local_sea_level_stations = pd.read_csv("data/clean_data/sea_level_station_data.csv", index_col=0)
# giss_temp = pd.read_csv("data/clean_data/temperature_monthly.csv", index_col=0)
# full_globe_temp = pd.read_csv("data/clean_data/temperature_yearly.csv", index_col=0, 
#                               parse_dates=True, squeeze=True, header=None, names=['year', 'mean temp'])

In [ ]:
giss_temp

## 7. Accessing data

For more details, see http://pandas.pydata.org/pandas-docs/stable/indexing.html

The general philosophy for accessing values inside a Pandas datastructure is that, unlike a numpy array that only allows to index using integers a Series allows to index with the values inside the index. That makes the code more readable.

### In a series

By default `[]` on a series accesses values using the index, not the location in the series.

In [ ]:
s = pd.Series(range(5), index=list("abcde"))
s

In [ ]:
s["d"]

So, what is the sea level rise in 1880?

In [ ]:
# This index is non-trivial (will talk more about these datetime 
# objects further down):
full_globe_temp.index.dtype

To easily make a `Timestamp` object from a string, one can use the `pd.to_datetime()` function:

In [ ]:
pd.to_datetime('1880')

In [ ]:
# By default [] on a series accesses values using the index, 
# not the location in the series
print(full_globe_temp[pd.to_datetime('1880')])
# print(temp1[0])  # This would fail!!

In [ ]:
# Another more explicit way to do the same thing is to use loc
print(full_globe_temp.loc[pd.to_datetime('1990')])
print(full_globe_temp.iloc[0], full_globe_temp.iloc[-1])

In [ ]:
# Year of the last record?
full_globe_temp.index[-1]

Pandas datastructures are extensible, new records can be added:

In [ ]:
full_globe_temp[pd.to_datetime('2011')] = np.nan

### In a dataframe

In 2D, same idea, though in a DF `[]` accesses columns (Series) instead.

In [ ]:
giss_temp["Jan"]

To make more complete queries, `.loc` and `.iloc` should be used: they are faster and less ambiguous to access individual values, slices or masked selections:

In [ ]:
giss_temp.loc[1979, "Dec"]

In [ ]:
# Slicing can be done with .loc and .iloc
print(giss_temp.loc[1979, "Jan":"Jun"])  # Note that the end point is included unlike NumPy!!!
print(giss_temp.loc[1979, ::2])

Masking can also be used in one or more dimensions. For example, another way to grab every other month for the first year:

In [ ]:
mask = [True, False] * 6
print(giss_temp.iloc[0, mask])
print(giss_temp.loc[1880, mask])

Just like a series, we could also add a new column like a new entry in a dictionary:

In [ ]:
giss_temp["totals"] = giss_temp.sum(axis=1)
giss_temp

Let's remove this new column, we will learn to do this differently

In [ ]:
giss_temp = giss_temp.drop("totals", axis=1)

More complex queries rely on the same concepts. For example what are the names, and IDs of the sea level stations in the USA?

In [ ]:
local_sea_level_stations.columns

In [ ]:
american_stations = local_sea_level_stations["Country"] == "USA"
american_stations

In [ ]:
local_sea_level_stations.loc[american_stations, ["ID", "Station Name"]]

In [ ]:
local_sea_level_stations

**EXERCISE:** What are the countries of the stations in latitudes above 60 and below -60. Print each country only once.

In [ ]:
# Your code here

## 8. Working with dates and times

More details at http://pandas.pydata.org/pandas-docs/stable/timeseries.html

We parsed the dates to build the `full_globe_temp` Series. Let's see what that allows us to do.

In [ ]:
# Its dtype is NumPy's new 'datetime64[ns]':
full_globe_temp.index

The advantage of having a real datetime index is that operations can be done efficiently on it. Let's add a flag to signal if the value is before or after the great depression's black Friday:

In [ ]:
black_friday = pd.to_datetime('1929-10-29')
full_globe_temp.index > black_friday

### Timestamps or periods?

In [ ]:
# Convert its index from timestamp to period: it is more meaningfull 
# since it was measured and averaged over the year...
full_globe_temp.index = full_globe_temp.index.to_period()
full_globe_temp

See also `to_timestamp` to conver back to timestamps and its `how` method to specify when inside the range to set the timestamp.

### Resampling

Another thing that can be done is to resample the series, downsample or upsample. Let's see the series converted to 10 year blocks or upscale to a monthly series:

In [ ]:
# Frequencies can be specified as strings: "us", "ms", "S", 
# "T", "H", "D", "B", "W", "M", "A", "3min", "2h20", ...
# More aliases at http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases
full_globe_temp.resample("M").mean()

In [ ]:
full_globe_temp.resample("10A").mean()

### Generating `DatetimeIndex` objects

The index for `giss_temp` isn't an instance of datetimes so we may want to generate such `DatetimeIndex` objects. This can be done with `date_range` and `period_range`:

In [ ]:
# Can specify a start date and a number of values desired. By default 
# it will assume an interval of 1 day:
pd.date_range('1/1/1880', periods=4)

In [ ]:
# Can also specify a start and a stop date, as well as a frequency
pd.date_range('1/1/1880', '1/1/2016', freq="A")

Note that `"A"` by default means the end of the year. Other times in the year can be specified with `"AS"` (start), `"A-JAN"` or `"A-JUN"`. Even more options can be imported from `pandas.tseries.offsets`:

In [ ]:
from pandas.tseries.offsets import YearBegin
pd.date_range('1/1/1880', '1/1/2015', freq=YearBegin())

Actually we will convert that dataset to a 1D dataset, and build a monthly index, so lets build a monthly period index

In [ ]:
giss_temp_index = pd.period_range('1/1/1880', '04/1/2015', freq="M")
giss_temp_index

## 9. Transforming datasets: rolling stats, custom transformations, sort and reshaping

For more details, see http://pandas.pydata.org/pandas-docs/stable/computation.html and http://pandas.pydata.org/pandas-docs/stable/reshaping.html

### Rolling statistics

Let's remove high frequency signal and extract the trend:

In [ ]:
full_globe_temp.plot()
full_globe_temp.rolling(window=10).mean().plot(figsize=LARGE_FIGSIZE)

**EXERCISE:** Refer to `exercises/pandas/pandas_moving_average/pandas_moving_average.py`

Let's look at our `local_sea_level_stations` dataset some more, to learn more about it and also do some formatting. What is the range of dates and lattitudes we have, the list of countries, the range of variations, ...

In [ ]:
# What about the range of dates?"
local_sea_level_stations["Date"].min(), local_sea_level_stations["Date"].max(), local_sea_level_stations["Date"].iloc[-1]

In [ ]:
local_sea_level_stations.dtypes

### Apply: transforming Series

We don't see the correct range of dates because the dates are of dtype "Object", (usually meaning strings). Let's convert that using `apply`:

In [ ]:
local_sea_level_stations["Date"].apply(pd.to_datetime)

This `apply` method is very powerful and general. We have used it to do something we could have done with `astype`, but any custom function can be provided to `apply`.

In [ ]:
local_sea_level_stations["Date"] = local_sea_level_stations["Date"].apply(pd.to_datetime)

# Now we can really compare the dates, and therefore get a real range:
print(local_sea_level_stations["Date"].min(), local_sea_level_stations["Date"].max())

Now that we know the range of dates, to look at the data, sorting it following the dates is done with `sort`:

In [ ]:
local_sea_level_stations.sort_values(by="Date")

Since many stations last updated on the same dates, it is logical to want to sort further, for example, by `Country` at constant date, and then by station name:

In [ ]:
local_sea_level_stations.sort_values(by=["Date", "Country", "Station Name"], ascending=False)

**EXERCISE:** Use the `apply` method to search through the stations names for a station in New York. What is the ID of the station?

In [ ]:
# Your code here

### Stack and unstack

Let's look at the GISS dataset differently. Instead of seeing the months along the axis 1, and the years along the axis 0, it would could be good to convert these into an outer and an inner axis along only 1 time dimension.

Stacking and unstacking allow to convert pandas structures from wide formats to long format. Here is an example:

In [ ]:
giss_temp

In [ ]:
giss_temp.stack?

In [ ]:
giss_temp_series = giss_temp.stack()
giss_temp_series.name = "Temp anomaly"
giss_temp_series

In [ ]:
# Note the nature of the result:
type(giss_temp_series)

### A side note: Multi-indexes

For more details, see http://pandas.pydata.org/pandas-docs/stable/advanced.html

In [ ]:
# Note the nature of the resulting index:
giss_temp_series.index

In [ ]:
# It is an index made of 2 columns. Let's fix the fact that one of them doesn't have a name:
giss_temp_series.index = giss_temp_series.index.set_names(["year", "month"])

In [ ]:
# We can now access deviations by specifying the year and month:
giss_temp_series[(1980, "Jan")]

But this new multi-index isn't very good, because is it not viewed as 1 date, just as a tuple of values:

In [ ]:
giss_temp_series.plot(figsize=LARGE_FIGSIZE)

To improve on this, let's reuse an index we generated above with `period_range`:

In [ ]:
giss_temp_series.index = giss_temp_index
giss_temp_series.plot(figsize=LARGE_FIGSIZE)

## 10. Data agregation and summarization

Now that we have a good grasp on our datasets, Let's transform and analyze them some more to prepare them to compare them. The 3 function(alities)s to learn about here are `groupby`, `merge` and `pivot_table`.

### GroupBy

For more details, see http://pandas.pydata.org/pandas-docs/stable/groupby.html

Let's first look at the average lattitude of sea level stations for each country in `local_sea_level_stations`:

In [ ]:
local_sea_level_stations

#### Grouping by an existing column

In [ ]:
sl_stations_grouped_country = local_sea_level_stations.groupby("Country")

What kind of object did we create?

In [ ]:
type(sl_stations_grouped_country)

What to do with that strange GroupBy object? We can first loop over it to get the labels and the sub-dataframes for each group:

In [ ]:
for group_name, subdf in sl_stations_grouped_country:
    print(group_name)
    print(subdf)
    print("")

Each subdf contains all the series since it is a `DataFrameGroupBy` object. In fact the groupby object can be reduced to one of these series with regular index:

In [ ]:
sl_stations_grouped_country["Lat"]

And that allows us to compute the mean latitudes, by applying the aggregation function to that sub object.

In [ ]:
sl_stations_grouped_country["Lat"].mean()

#### Grouping by the index

Let's explore the sea levels, splitting into calendar years to compute average sea levels for each year. That also shows that we can do the groupby on the index:

In [ ]:
mean_sea_level

In [ ]:
sl_grouped_year = mean_sea_level.groupby(int)

Something else that can be done with such an object is to look at its `groups` attribute to see the labels mapped to the rows involved:

In [ ]:
sl_grouped_year.groups

How to aggregate the results of this grouping depends on what we want to see: do we want to see averaged over the years? That is so common that it has been implemented directly as a method on the `GroupBy` object.

In [ ]:
sl_grouped_year.mean()

In [ ]:
# We can apply any other reduction function or even a dict of functions 
# using aggregate:
sl_grouped_year.aggregate({"mean_global": np.std})

#### `GroupBy` to transform different parts of a DF differently

Another possibility is to transform each group separately, rather than aggregate. For example, here we group the mean sea levels over decades and subtract to each value the average over that decade:

In [ ]:
sl_grouped_decade = mean_sea_level.groupby(lambda x: int(x/10.))
sl_grouped_decade.groups.keys()

In [ ]:
def normalize(subframe):
    return (subframe - subframe.mean()) / subframe.std()

In [ ]:
sl_grouped_decade.transform(normalize)

### SQL-type `join`

For more details, see http://pandas.pydata.org/pandas-docs/stable/merging.html

We would like to add more information to our station table. In particular, we would like to add a column of continents, so that we can group data by continent. That can be done using a table of country code and continent code found at http://dev.maxmind.com/geoip/legacy/codes/country_continent/. Unfortunately, the country codes we have are 3-letter but the mapping between 3-letter and 2-letter code can be found in https://en.wikipedia.org/wiki/ISO_3166-1.

First let's load the 2 tables that together contain all the information needed. The country_continent table is stored locally in the `data/misc/` folder:

In [ ]:
country2_continent = pd.read_csv("data/misc/country_continent.csv", 
                                 na_values=["--"], keep_default_na=False)
country2_continent

In [ ]:
# Let's check we have found stations on all continents
country2_continent["continent code"].unique()

**Quiz**: What happens if `keep_default_na` is not set in the `read_csv` above?

In [ ]:
# Local backup:
# country_codes = pd.read_excel("data/misc/country_codes.xlsx")
tables = pd.read_html("https://en.wikipedia.org/wiki/ISO_3166-1")
country_codes = tables[0]
country_codes

**Exercise** The table's headers are found in the first row. Make them the proper headers.

In [ ]:
# Your solution here

In [ ]:
# Don't look! solution below...

































In [ ]:
new_headers = list(country_codes.iloc[0])
country_codes.columns = new_headers
country_codes = country_codes.drop(0)
country_codes

From these 2 tables, let's first make a table with the 3-letter county codes together with the continents. That's a `join` operation which can be done in Pandas with `pd.merge`:

In [ ]:
pd.merge?

In [ ]:
country_code_cols = ["English short name (upper/lower case)", 
                     "Alpha-2 code", "Alpha-3 code"]
raw_mapping = pd.merge(country_codes[country_code_cols], 
                       country2_continent, 
                       how="outer", 
                       left_on="Alpha-2 code", 
                       right_on="iso 3166 country")
raw_mapping

Really, all we are after is the mapping between the 3-letter code and the continent, so let's drop everything else:

In [ ]:
mapping = raw_mapping[["Alpha-3 code", "continent code"]].dropna(how="any")
mapping

In [ ]:
merged = pd.merge(local_sea_level_stations, mapping, how="left", 
                  left_on="Country", right_on="Alpha-3 code")
merged

In [ ]:
local_sea_level_stations = merged.drop("Alpha-3 code", axis=1)

**Exercise**: Use `groupby` to compute the number of stations in each continent.

In [ ]:
# Your code here.

### Pivot_table

For more details, see http://pandas.pydata.org/pandas-docs/stable/reshaping.html

Pivot table also allows to summarize the information, allowing to convert repeating column values into dimensions of a new table. 

For example, let's say that we would like to know how many sea level stations are on various continents. And we would like to group the answers into 2 categories: the stations that have been updated recently (after `2000/1/1`) and the others. 

Thanks to the previous merge, we already have a column with the (repeating) continent codes:

In [ ]:
local_sea_level_stations["continent code"].unique()

These values will become the index of our future table. The columns of our future table should have 2 values, whether the station was updated recently or not. Let's build a column to store that information:

In [ ]:
local_sea_level_stations["Recently updated"] = local_sea_level_stations["Date"] > pd.to_datetime("2000")

Finally, what value will be displayed inside the table? The values should be extracted from a column, `pivot_table` allowing an aggregation function to be applied when more than 1 value is found for a given case. Each station should count for 1, and we could aggregate multiple stations by summing these 1s:

In [ ]:
local_sea_level_stations["Number of stations"] = np.ones(len(local_sea_level_stations))

In [ ]:
station_counts = pd.pivot_table(local_sea_level_stations, 
                                index="Recently updated", 
                                columns="continent code", 
                                values="Number of stations", 
                                aggfunc=np.sum)
station_counts

**EXERCISE (Optional):** How would we build the same dataframe with a `groupby` operation?

In [ ]:
# Your code here

**EXERCISE:** 
  1. Build a similar table but with the `recently updated` information along the columns, and all country codes as rows. 
  2. How many recently updated stations is there in the world? Not recently updated stations? Look at the documentation for `pd.pivot_table`'s margins option. Which country has the most stations?
  3. Build another table showing the `recently updated` information along the columns, and all continents as rows but with both the average latitude and average longitude as the values.

In [ ]:
# Your code here

**EXERCISE:** Refer to `exercises/pandas/pandas_dataframe/pandas_dataframe.py`

## 11. Correlations and regressions

### Correlation coefficients

Both Series and dataframes have a `corr` method to compute the correlation coefficient between series:

In [ ]:
# Let's see what how the various sea levels are correlated with each other:
mean_sea_level["northern_hem"].corr(mean_sea_level["southern_hem"])

In [ ]:
# If series are already grouped into a DataFrame, computing all 
# correlation coeff is trivial:
mean_sea_level.corr()

Note: by default, the method used is the `Pearson` correlation coefficient. Other methods are available (`kendall`, `spearman` using the `method` kwarg).

In [ ]:
# Visualize the correlation matrix
plt.imshow(mean_sea_level.corr(), interpolation="nearest", cmap=plt.cm.Blues)

In [ ]:
# let's make it a little better to confirm that learning about global sea level cannot be done from just 
# looking at stations in the northern hemisphere:
plt.imshow(mean_sea_level.corr(), interpolation="nearest", cmap=plt.cm.Blues)
plt.xticks(np.arange(3), mean_sea_level.corr().columns)
plt.yticks(np.arange(3), mean_sea_level.corr().index)
plt.colorbar()

### Series modelling with `OLS`

Pandas's ability to build Ordinary Least Square modelling relies on the `statsmodels` package. 

OLS in pandas requires to pass a `y` series and an `x` series to do a fit of the form `y ~ x`. But the formula can be more complex by providing a `DataFrame` for x and reproduce a formula of the form `y ~ x1 + x2`. 

Also possible are rolling and expanding OLS.

In [ ]:
from pandas.stats.api import ols

In [ ]:
model = ols(y=mean_sea_level["mean_global"], 
            x=mean_sea_level[["northern_hem", "southern_hem"]])
model

In [ ]:
model.summary_as_matrix

In [ ]:
plt.figure(figsize=LARGE_FIGSIZE)
mean_sea_level["mean_global"].plot()
model.predict().plot(label="OLS prediction")
plt.legend(loc="upper left")

### An interlude: data alignment

#### Converting the floating point date to a timestamp

Now, we would like to look for correlations between our monthly temperatures and the sea levels we have. For this to be possible, some data alignment must be done since the time scales are very different for the 2 datasets. 

In [ ]:
mean_sea_level["mean_global"].index

In [ ]:
giss_temp_series.index

We will need a function that converts the floating point dates in the sea level to timestamps:

In [ ]:
import calendar
def floating_year_to_timestamp(float_date):
    """ Convert a date as a floating point year number to a pandas timestamp object.
    """
    year = int(float_date)
    days_per_year = 366 if calendar.isleap(year) else 365
    remainder = float_date - year
    daynum = 1 + remainder * (days_per_year - 1)
    daynum = int(round(daynum))
    
    # Convert day number to month and day
    day = daynum
    month = 1
    while month < 13:
        month_days = calendar.monthrange(year, month)[1]
        if day <= month_days:
            return pd.to_datetime(str(year)+"/"+str(month)+"/"+str(day))
        day -= month_days
        month += 1
    raise ValueError('{} does not have {} days'.format(year, daynum))

In [ ]:
dt_index = pd.Series(mean_sea_level["mean_global"].index).apply(floating_year_to_timestamp)
dt_index

In [ ]:
mean_sea_level = mean_sea_level.reset_index(drop=True)
mean_sea_level.index = dt_index
mean_sea_level

Now, how to align the 2 series?

### Aligning `Series` and `DataFrame`s with the `align` method

Let's downscale the frequency of the sea levels to monthly, like the temperature reads:

In [ ]:
monthly_mean_sea_level = mean_sea_level.resample("MS").mean().to_period()
monthly_mean_sea_level

Now that the series are using the same type and frequency of indexes, to align them is trivial:

In [ ]:
monthly_mean_sea_level["mean_global"].align(giss_temp_series)

By default, the alignment leads to the *union* of the 2 indexes. We would rather have the intersection to avoid so much missing data:

In [ ]:
monthly_mean_sea_level["mean_global"].align(giss_temp_series, join='inner')

Let's store that result:

In [ ]:
aligned_sl, aligned_temp = monthly_mean_sea_level["mean_global"].align(giss_temp_series, 
                                                                       join='inner')
aligned_df = pd.DataFrame({"mean_sea_level": aligned_sl, "mean_global_temp": aligned_temp})

The alignment can even be done on an entire dataframe:

In [ ]:
monthly_mean_sea_level.align(giss_temp_series, axis=0, join='inner')

In [ ]:
aligned_sea_levels, aligned_temp = monthly_mean_sea_level.align(giss_temp_series, axis=0, join='inner')
aligned_monthly_data = aligned_sea_levels.copy()
aligned_monthly_data["global_temp"] = aligned_temp
aligned_monthly_data

### Correlations between sea levels and temperatures

In [ ]:
aligned_monthly_data.plot(figsize=LARGE_FIGSIZE)

In [ ]:
aligned_monthly_data.corr()

In [ ]:
model = ols(y=aligned_monthly_data["southern_hem"], 
            x=aligned_monthly_data["global_temp"])
model.r2

What if we had done the analysis yearly instead of monthly to remove seasonal variations?

In [ ]:
aligned_yearly_data = aligned_monthly_data.resample("A").mean()
aligned_yearly_data.plot()

In [ ]:
aligned_yearly_data.corr()

In [ ]:
model = ols(y=aligned_yearly_data["southern_hem"], 
            x=aligned_yearly_data["global_temp"])
model.r2

**Exercise (Optional)**: If not already done, load the greenhouse gaz data located at `data/greenhouse_gaz/co2_mm_global.txt` and add it to the `aligned_monthly_data` dataframe. Use that to plot the greenhouse data together with the other curves for sea level and temperature. How well is this dataset correlated with temperature and sea level data?

In [ ]:
# you code here

## 12. Predictions from auto regression models

An auto-regresssive model fits existing data and builds a (potentially predictive) model of the data fitted. We use the timeseries analysis (`tsa`) submodule of `statsmodels` to make out-of-sample predictions for the upcoming decades:

In [ ]:
import statsmodels as sm
# Let's remove seasonal variations by resampling annually
data = giss_temp_series.resample("A").mean().to_timestamp()
ar_model = sm.tsa.ar_model.AR(data, freq='A')
ar_res = ar_model.fit(maxlag=60, disp=True)

In [ ]:
plt.figure(figsize=LARGE_FIGSIZE)
pred = ar_res.predict(start='1950-1-1', end='2070')
data.plot(style='k', label="Historical Data")
pred.plot(style='r', label="Predicted Data")
plt.ylabel("Temperature variation (0.01 degC)")
plt.legend()

**EXERCISE:** Make another auto-regression on the sea level of the Atlantic ocean to estimate how much New York is going to flood in the coming century. To do that, extract the ID of a station in NewYork from the `local_sea_level_stations` dataset, and use it to download timeseries in NY. Reminder: the URL to build is for the form http://www.psmsl.org/data/obtaining/met.monthly.data/[ID].metdata once the ID is known.

In [ ]:
# Your code here

## Want to practice more?

**EXERCISE (computations):** Refer to `exercises/stock_returns/stock_returns.py`

**EXERCISE (stats, groupby, timeseries):** Refer to `exercises/pandas_wind_statistics/pandas_wind_statistics.py`